In [1]:
import scrapy
from scrapy.http import TextResponse


In [11]:
url = "https://movie.naver.com/movie/running/current.nhn"
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding = "utf-8")
links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
links = [response.urljoin(link) for link in links]
links[-3:]

['https://movie.naver.com/movie/bi/mi/basic.nhn?code=52555',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=11757',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=11093']

In [15]:
link = links[0]
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189537'

In [16]:
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding = "utf-8")
response

<200 https://movie.naver.com/movie/bi/mi/basic.nhn?code=189537>

In [22]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()')[0].extract()

count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()')[0].extract()

star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()

star = "".join(star)

title, count, star

('#살아있다', '1,291,102명', '7.97')

In [23]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory '/Users/hyunjoon/opt/anaconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/hyunjoon/Documents/dss/naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


In [25]:
%%writefile naver_movie/naver_movie/items.py

import scrapy

class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()
    link = scrapy.Field()


Overwriting naver_movie/naver_movie/items.py


In [37]:
%%writefile naver_movie/naver_movie/spiders/spider.py

import scrapy

from naver_movie.items import NaverMovieItem

class Spider(scrapy.Spider):
    
    name = 'NaverMovieItem'
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback = self.parse_content)
            
    def parse_content(self, response):
        item =NaverMovieItem()
        item['title'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()')[0].extract()
        try:
            item["count"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p/text()')[0].extract()
        except:
            star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
      
        item["star"] = "".join(star)
        item["link"] = response.url
        yield item
        
    

Overwriting naver_movie/naver_movie/spiders/spider.py


In [38]:
!sed -i .bak 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py

In [39]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovieItem

Overwriting run.sh


In [40]:
!source run.sh

2020-07-03 18:12:22 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: naver_movie)
2020-07-03 18:12:22 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-18.7.0-x86_64-i386-64bit
2020-07-03 18:12:22 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-03 18:12:22 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'naver_movie',
 'NEWSPIDER_MODULE': 'naver_movie.spiders',
 'SPIDER_MODULES': ['naver_movie.spiders']}
2020-07-03 18:12:22 [scrapy.extensions.telnet] INFO: Telnet Password: 47f39458e20f9b97
2020-07-03 18:12:22 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

In [ ]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()')[0].extract()

count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()')[0].extract()

star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()

star = "".join(star)

title, count, star